# 二维卷积（互相关运算） `corr2d()`
- 卷积之后，输出大小等于输入大小$n_h \times n_w$减去卷积核大小$k_h \times k_w$，即：
  - $$(n_h-k_h+1) \times (n_w-k_w+1).$$

- 我们在`corr2d`函数中实现二维卷积运算
  - 该函数接受输入张量`X`和卷积核张量`K`，并返回输出张量`Y`。

In [52]:
import torch
from torch import nn
from d2l import torch as d2l

In [53]:
def corr2d(X,K):
    """"计算二维互相关运算"""
    h, w = K.shape # h ,w 保存卷积核大小
    Y = torch.zeros((X.shape[0] -h +1 ,X.shape[1]-w + 1)) # 初始化输出矩阵，全为0
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):# Y中每个元素都做一次互相关运算
            Y[i,j] = (X[i:i+h,j:j+w]*K).sum() # i,j元素值=取此处核大小个矩阵，与K做点积，求和
    return Y

- 验证一下上面的函数

In [54]:
X = torch.tensor([[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]])
K = torch.tensor([[0.0,1.0],[2.0,3.0]])
corr2d(X,K)

tensor([[19., 25.],
        [37., 43.]])

# 自己写个卷积层

- 基于上面定义的`corr2d`函数[**实现二维卷积层**]。
- 在`__init__`构造函数中，将`weight`和`bias`声明为两个模型参数。
  - 这两个也是需要训练的参数，用`nn.Parameter()`声明,同时进行初始化
- 前向传播函数调用`corr2d`函数并添加偏置。

In [55]:
class Conv2D(nn.Module):
    def __init__(self,kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))
    def forward(self,x):
        return corr2d(x,self.weight) + self.bias

- 高度和宽度分别为$h$和$w$的卷积核可以被称为$h \times w$卷积或$h \times w$卷积核。
- 我们也将带有$h \times w$卷积核的卷积层称为$h \times w$卷积层。

# 应用一下：检测图像中的边缘
- 自己构造一个黑白图像

In [56]:
X = torch.ones((6,8))
X[:,2:6] = 0
X

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

- 自己构造一个高度1，宽度2 的卷积核K
- 当进行互相关运算时，如果水平相邻的两元素相同，则输出为0，否则输出为非0

In [57]:
K = torch.tensor([[1.0,-1.0]])

- 现在，我们对参数`X`（输入）和`K`（卷积核）执行互相关运算。
- 如下所示，[**输出`Y`中的1代表从白色到黑色的边缘，-1代表从黑色到白色的边缘**]，其他情况的输出为$0$。


In [58]:
Y = corr2d(X,K)
Y # Y得到输出，即得到边缘。非0值表示边缘

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

- 如果图像转置之后，用这个卷积核，会发现不行
- 这个[**卷积核`K`只可以检测垂直边缘**]，无法检测水平边缘。

In [59]:
corr2d(X.t(),K)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

# 上面是自己设置的一个卷积核，接下来自己来学习出卷积核
- 当不知道卷积核数值应该为多少时，就要通过学习得到
- 卷积核大小和维度，大多依据经验，或者多组实验然后取最好的

In [60]:
# 构造一个二维卷积层，它具有1个输出通道和形状为(1,2)的卷积核
conv2d = nn.Conv2d(1,1,kernel_size = (1,2),bias=False)

# 这个二维卷积层使用四维输入和输出格式（批量大小、通道、高度、宽度）
# 其中批量大小和通道数都为1
X = X.reshape((1,1,6,8))
Y = Y.reshape(1,1,6,7) # 输出y的大小又公式计算得出
lr = 3e-2 # 学习率

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat -Y )**2
    conv2d.zero_grad()
    l.sum().backward()
    # 迭代卷积核,即手动更新参数，这里没有bias
    conv2d.weight.data[:] -= lr*conv2d.weight.grad
    if (i+1) % 2 == 0 :
        print(f"epoch{i+1},loss{l.sum():.3f}")


epoch2,loss7.514
epoch4,loss1.637
epoch6,loss0.429
epoch8,loss0.135
epoch10,loss0.049


- 在$10$次迭代之后，误差已经降到足够低。现在我们来看看我们[**所学的卷积核的权重张量**]。

In [61]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9657, -1.0089]])

# 多输入通道 从零实现
- 即每个输入通道执行互相关操作，然后将结果相加

In [62]:
import torch
from d2l import torch as d2l
def corr2d_multi_in(X,K):
    # 先遍历X和K的第0个维度（通道维度），再把他们加在一起
    return sum(d2l.corr2d(x,k) for x,k in zip(X,K))

- 验证一下

In [63]:
X = torch.tensor([[[0.0,1.0,2.0],[3.0,4.0,5.0],[6.0,7.0,8.0]],
                  [[1.0,2.0,3.0],[4.0,5.0,6.0],[7.0,8.0,9.0]]])

K = torch.tensor([[[0.0,1.0],[2.0,3.0]],
                  [[1.0,2.0],[3.0,4.0]]])

corr2d_multi_in(X,K)

tensor([[ 56.,  72.],
        [104., 120.]])

# 多通道输出
- $c_i$和$c_o$分别表示输入和输出通道的数目
- 如下所示，我们实现一个[**计算多个通道的输出的互相关函数**]。

In [64]:
def corr2d_multi_in_out(X,K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算
    # 最后将所有结果都叠加在一起 
    # torch.stack实现 ：按第0维度叠加一起
    return torch.stack([corr2d_multi_in(X,k) for k in K],0)

- 通过将核张量`K`与`K+1`（`K`中每个元素加$1$）和`K+2`连接起来，构造了一个具有$3$个输出通道的卷积核。
- 三个卷积核

In [65]:
K = torch.stack((K,K+1,K+2),0)
K.shape ,K

(torch.Size([3, 2, 2, 2]),
 tensor([[[[0., 1.],
           [2., 3.]],
 
          [[1., 2.],
           [3., 4.]]],
 
 
         [[[1., 2.],
           [3., 4.]],
 
          [[2., 3.],
           [4., 5.]]],
 
 
         [[[2., 3.],
           [4., 5.]],
 
          [[3., 4.],
           [5., 6.]]]]))

- 三个卷积核-》三个输出
- 下面，我们对输入张量`X`与卷积核张量`K`执行互相关运算。现在的输出包含$3$个通道，第一个通道的结果与先前输入张量`X`和多输入单输出通道的结果一致。

In [66]:
corr2d_multi_in_out(X,K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

# 1x1卷积核
- 用的挺多
- 不识别空间信息
  - 因为只看每个通道的一个像素，不看周围的
- 只融合通道，进行降维
  - 不同通道，每个像素，按照1x1卷积核对应的数值（权值），进行加权求和的运算
- 相当于全连接层
  - 输入是，每个通道的所有像素（可以把每个通道拉成一维向量）
  - 权重为 维度为输入通道数x输出通道数的矩阵
    - 矩阵的值是卷积核的值

# 小结
* 多输入多输出通道可以用来扩展卷积层的模型。
* 当以每像素为基础应用时，$1\times 1$卷积层相当于全连接层。
* $1\times 1$卷积层通常用于调整网络层的通道数量和控制模型复杂性。